In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import time
from model import *
from data import *

![U-NET](https://img-blog.csdn.net/20170417204711784?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvdTAxMjkzMTU4Mg==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)


U-Net，基於Fully Convolutional Networks(全捲積網路)，輸入和輸出都是圖，沒有全連接層。


## Train your Unet with membrane data
membrane data is in folder Membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [ ]:
tic = time.time()
BatchSize = 1

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(BatchSize,'Membrane','image','label',data_gen_args,save_to_dir = None)
#myGene = trainGenerator(BatchSize,'Fixed','image','label',data_gen_args,save_to_dir = None)
#myGene = trainGenerator(BatchSize,'Mitochondrion','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('UNet_Membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model_checkpoint = ModelCheckpoint('UNet_Fixed.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model_checkpoint = ModelCheckpoint('UNet_Mitochondrion.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=10,callbacks=[model_checkpoint])

toc = time.time()
print("Time=" + str((toc-tic)) + "sec")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


C:\Users\1203087\Documents\UNet512_TrainForClass\model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Epoch 1/10
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
   8/2000 [..............................] - ETA: 39:28:23 - loss: 0.5089 - acc: 0.7171

### Train with npy file

In [ ]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [ ]:
testGene = testGenerator("Fixed/test")
model = unet()
model.load_weights("UNet_Fixed.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("Fixed/test",results)